In [1]:
import numpy as np
import pandas as pd

In [2]:
data = np.array(pd.read_csv("Dataset/Text_Emotion_Data.csv"))
stopwords = np.array(pd.read_csv("Dataset/stopwords.txt", sep=" ", header=None)).flatten()

In [3]:
X = data[:, 0]
y = data[:, 1]

In [4]:
import re

def remove_none_alpha(x):
    regex = re.compile('[^a-zA-Z ]')
    return regex.sub('', x)

In [5]:
stopwords = [remove_none_alpha(w) for w in stopwords]

In [6]:
X_seq = []
for sentence in X:
    words_seq = remove_none_alpha(sentence).lower().split(" ")
    words_seq = [w for w in words_seq if len(w) > 2]
    words_seq = [w for w in words_seq if w not in stopwords]
    X_seq.append(words_seq)

In [7]:
max_len = len(max(X_seq, key=len))

# create a new list of lists with the desired size, padding with empty strings
sequences = [['' for _ in range(max_len)] for _ in range(len(X_seq))]

# copy the elements from the original lists into the new list, padding with empty strings as needed
for i, sequence in enumerate(X_seq):
    sequences[i][:len(sequence)] = sequence

In [8]:
import numpy as np

def one_hot_encode(sequence, word_to_idx):
    num_words = len(word_to_idx)
    encoding = np.zeros((len(sequence), num_words))
    for i, word in enumerate(sequence):
        if word in word_to_idx:
            encoding[i, word_to_idx[word]] = 1
    return encoding

def one_hot_encode2(sequence, word_to_idx, max_bit_num):
    encoding = np.zeros((len(sequence), max_bit_num))
    for i, word in enumerate(sequence):
        if word in word_to_idx:            
            binary_string = bin(word_to_idx[word])[2:]  # exclude the "0b" prefix
            binary_array = np.array([int(digit) for digit in binary_string])
            binary_array = np.pad(binary_array, (0, max_bit_num - len(binary_array)), 'constant')

            encoding[i, :] = binary_array
    return encoding

unique_words = list(set([word for sequence in sequences for word in sequence]))
word_to_idx = {word: idx for idx, word in enumerate(unique_words)}

max_bit_num = 1
while(True):
    if(2**max_bit_num > len(unique_words)): break
    max_bit_num += 1

encoded_sequences = []
for sequence in sequences:
    encoded_sequences.append(one_hot_encode2(sequence, word_to_idx, max_bit_num))

encoded_sequences = np.array(encoded_sequences)

In [9]:
X_train = encoded_sequences[:-150]
X_test = encoded_sequences[-150:]
# y_train_str = y[:-150]
# y_test_str = y[-150:]

In [10]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
y_encoded = le.fit_transform(y)

In [11]:
# Define the function you want to apply
def to_binary(num):
    binary_string = bin(num)[2:]  # exclude the "0b" prefix
    binary_array = np.array([int(digit) for digit in binary_string])
    binary_array = np.pad(binary_array, (0, 3 - len(binary_array)), 'constant')
    return binary_array

# Create a NumPy array
arr = np.array(y_encoded)

# Initialize the result array
y_encoded_onehot = np.empty((arr.shape[0], 3), dtype=int)

# Apply the function on all elements of the array
for i, num in enumerate(arr):
    y_encoded_onehot[i] = to_binary(num)


In [12]:
y_train = y_encoded_onehot[:-150]
y_test = y_encoded_onehot[-150:]

In [13]:
import tensorflow as tf

# Define the Elman network architecture
input_dim = 13
hidden_dim = 100
output_dim = 3
num_steps = 19

inputs = tf.keras.layers.Input(shape=(num_steps, input_dim))
hidden_layer = tf.keras.layers.SimpleRNN(hidden_dim, activation='s')(inputs)
outputs = tf.keras.layers.Dense(output_dim, activation='softmax')(hidden_layer)

# Define the model
model = tf.keras.Model(inputs=inputs, outputs=outputs)

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=optimizer, loss='mse', metrics=[tf.metrics.Accuracy()])

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32)


Epoch 1/100
97/97 [==============================] - 3s 5ms/step - loss: 0.1316 - accuracy: 0.0000e+00
Epoch 2/100
97/97 [==============================] - 0s 4ms/step - loss: 0.1144 - accuracy: 0.0000e+00
Epoch 3/100
97/97 [==============================] - 0s 4ms/step - loss: 0.1138 - accuracy: 0.0000e+00
Epoch 4/100
97/97 [==============================] - 0s 4ms/step - loss: 0.1128 - accuracy: 0.0000e+00
Epoch 5/100
97/97 [==============================] - 0s 4ms/step - loss: 0.1106 - accuracy: 0.0000e+00
Epoch 6/100
97/97 [==============================] - 0s 4ms/step - loss: 0.1083 - accuracy: 0.0000e+00
Epoch 7/100
97/97 [==============================] - 0s 4ms/step - loss: 0.1092 - accuracy: 0.0000e+00
Epoch 8/100
97/97 [==============================] - 0s 4ms/step - loss: 0.1099 - accuracy: 0.0000e+00
Epoch 9/100
97/97 [==============================] - 0s 4ms/step - loss: 0.1112 - accuracy: 0.0000e+00
Epoch 10/100
97/97 [==============================] - 0s 4ms/step - loss:

In [14]:
import numpy as np

# Evaluate the model on the test data
loss = model.evaluate(X_test, y_test, verbose=0)

# Get the model's predictions on the test data
predictions = model.predict(X_test, verbose=0)

# Convert predictions and labels to class indices
predicted_labels = np.argmax(predictions, axis=1)

# Compute accuracy
accuracy = np.mean(predicted_labels == y_test)

print("Accuracy:", accuracy)


Accuracy: 0.0


C:\Users\AbbasMn\AppData\Local\Temp\ipykernel_26516\133833136.py:13: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  accuracy = np.mean(predicted_labels == y_test)


In [15]:
print(predicted_labels)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0]
